<a href="https://colab.research.google.com/github/pybuttheavy/Prog2kakushin/blob/main/Work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#csvから入力されたデータを、コマンド入力　”ファイル名　データ絞込条件　演算指示”　に応じて演算
import pandas as pd
import glob
import os

# 入力内容確認＋修正
def yorn():
  while True:
    a=input(">>")
    b=input(a+":これで間違いありませんか？     y / other")
    if b=="y" or b=="yes" or b=="":
      break
    else:
      print("再入力",end="")
  return a


def slash(strs:str):
  rlist=[]
  strs=strs.replace(" ","")
  while True:
    a=strs.find("\\")
    if a==-1:
      break
    elif a==0:
      strs=strs[1:]
      continue
    elif strs[-1]=="\\":
      strs=strs[:-1]
      continue
    else:
      c1=strs[:a]
      if c1!="":
        rlist.append(c1)
        strs=strs[a+1:]
  rlist.append(strs)
  return rlist

def emfil0(a):
  if a<10:
    return "0"+str(a)
  else:
    return str(a)

def p_help():
  print("コマンド一覧")
  print("help                               このコマンド一覧を表示します")
  print("exit                               アプリを終了します")
  print("input \http...A.csv \B.scv         指定したウェブ上のファイル'A.csv'を読み込み、'B.csv'として保存します")
  print("                                   指定していない引数がある場合は後で指定します")
  print("ls                                 保存しているcsvファイルリストを表示します")
  print("base \A.csv                        指定したcsvファイルを基に新しいcsvファイルを作成します　※注意　上書き機能、行、列追加機能未実装")
  print("make                               0からcsvファイルを作成、保存します　行名、列名やデータはあとで入力します  ※注意　未実装")
  print("manual                             自分でPythonコードを入力して実行します")
  print("del                                指定したcsvファイルを削除します")

def dfretuchushyu(df):
  # 列名指定
    retulist=[]
    while True:
      print("列の名前で指定（空白で終了）:")
      sibori=yorn()
      if sibori=="":
        break
      elif sibori in df.columns:
        retulist.append(sibori)
        continue
      else:
        print("エラー 列名'"+str(sibori)+"' が見つかりません")
        continue
    if retulist[0]!="":
      dfn=df[retulist]
    return dfn

def dfretudel(df):
  # 列名指定
    retulist=[]
    while True:
      print("列の名前で指定（空白で終了）:")
      sibori=yorn()
      if sibori=="":
        break
      elif sibori in df.columns:
        retulist.append(sibori)
        continue
      else:
        print("エラー 列名'"+str(sibori)+"' が見つかりません")
        continue
    if retulist[0]!="":
      for retu in retulist:
        dfn=df.drop(retu,axis=1)
    return dfn

def dfsuchihikaku(df):
  # 列の条件式指定(数値)
    print("列に対する条件式を指定（数値の大小を判定）")
    # 判定する列の指定
    while True:
      print("列名（空白で数値比較による列の抽出をスキップ) ：")
      retujoken=yorn()
      if retujoken not in df.columns and retujoken!="":
        print("エラー 列名'"+retujoken+"' が見つかりません")
        continue
      else:
        break
    if retujoken is not None:
      # 比較演算子の指定
      while True:
        hikakuenzan=input("> or < or >= or <=")
        if hikakuenzan in ["<",">","<=",">="]:
          break
        else:
          print("> < >= <= のいずれかを入力してください")
      # 比較対象（数値）を指定
      while True:
        try:
          hikakuatai=float(input("比較値"))
          break
        except:
          print("数を入力してください")
      if hikakuenzan==">":
        df1=df[retujoken]>hikakuatai
      elif hikakuenzan=="<":
        df1=df[retujoken]<hikakuatai
      elif hikakuenzan==">=":
        df1=df[retujoken]>=hikakuatai
      else:
        df1=df[retujoken]<=hikakuatai
      return df1
    else:
      return None

def dfstrhikaku(df):
  # 列の条件式指定（文字列・bool）
    print("列に対する条件式を指定（str型またはbool型が一致するか判定）")
    while True:
      print("列名（空白で文字列、bool比較による列の抽出をスキップ) ：")
      retujoken2=yorn()
      if retujoken2 not in df.columns and retujoken2!="":
        print("エラー 列名'"+retujoken2+"' が見つかりません")
        continue
      else:
        break
    print("比較値 文字列または　True または False")
    hikakumoji=yorn()

    if hikakumoji.lower()=="true":
      df1=[df[retujoken2]==True]
    elif hikakumoji.lower()=="false":
      df1=[df[retujoken2]==False]
    else:
      df1=[df[retujoken2]==hikakumoji]
    if retujoken2=="":
      return None
    else:
      return df1




# ここからプログラム本体
while True:
  # 最新のcsvファイルリスト入手
  csv_list=glob.glob("**/*.csv",recursive=True)
  #print(csv_list)

  # コマンド入力 小文字化 空白消し
  c=input("コマンド入力...>>")
  c=c.lower()
  c=c.replace(" ","")

  # コマンド解釈 /毎に区切り、comリストに入れる
  com=slash(c)

  # コマンドがhelpならコマンド一覧表示
  if com[0]=="help":
    p_help()
    continue

  # コマンドが'exit'ならばループ終了
  elif com[0]=="exit":
    break

  # input
  elif com[0]=="input":
    if len(csv_list)>=99:
      print("これ以上csvファイルは保存できません")
      continue

    # 引数が全てあるなら最初から二つを順に引数とみなす
    if len(com)>=3:
      url=com[1]
      uname=com[2]
    else:

      # 引数が一つもなければurlを入力させる　一つだけあればそれを第一引数とみなす
      if len(com)==1:
        url=print("ファイルのURLリンクを入力（'http～'で始まる）>>")
        url=yorn()
      else:
        url=com[1]

      # uname 保存するファイル名を入力させる　空白なら自動で名前を決定
      print("保存するファイルの名前を入力（末尾が'.csv'、空白で自動的に設定）>>")
      uname=yorn()
      if uname=="":
          a=1
          while emfil0(a)+".csv" in csv_list:
            a+=1
          uname=emfil0(a)+".csv"
      # url uname 指定ここまで

    # .csvでなければ修正(uname)
    while uname[-4:]!=".csv" or uname in csv_list:
        print("警告：保存するファイルの名前には未使用のcsvファイルを指定してください  1:現在の指定したファイル名に'.csv'を付け足す　その他:修正する")
        print("現在の指定ファイル名:"+uname)
        print("現在使われているファイル名")
        print(csv_list)
        unamere=input(">>")
        if unamere=="1":
          uname+=".csv"
        else:
          print("ファイル名を入力（末尾が'～.csv'）>>")
          uname=yorn()
    print("ファイル "+url+" を "+uname+" として保存します")
    try:
      !wget $url -o $uname
    except:
      print("エラー　ファイルが読み込めません")
      continue

    continue
    # input コマンド　ここまで

  # lsコマンド 行列数＋ファイル名
  elif com[0]=="ls":
    for n in csv_list:
      df=pd.read_csv(n)
      a=str(len(df.index))+"*"+str(len(df.columns))
      b=10-len(a)
      if b<1:
        b=1
      print(a+" "*b+n)

  # baseコマンド
  elif com[0]=="base":
    if len(com)==1 or com[1] not in csv_list :
      print("エラー")
      continue
    df=pd.read_csv(com[1])
    df3=df
    while True:
      print(str(len(df.columns))+"列",end="")
      print(df.columns)
      df2=df3
      print("データを絞り込みます    1:数値の大小で比較　2:値の一致で比較　3:列名で指定した列を抽出  4:列名で指定した列を除いて抽出  その他:抽出終了")
      chsibori=yorn()
      if chsibori=="1":
        # 列の条件式指定(数値大小比較)
        jokendf1=dfsuchihikaku(df2)
        df3=df2[jokendf1]
        df2=df3
      elif chsibori=="2":
        # 列の条件式指定(str型またはbool型一致)
        jokendf1=dfstrhikaku(df2)
        df3=df2[jokendf1]
        df2=df3
      elif chsibori=="3":
        # 列名指定抽出
        df3=dfretuchushyu(df2)
        df2=df3
        # 列名指定削除
      elif chsibori=="4":
        df3=dfretudel(df2)
        df2=df3
      else:
        break

    print("抽出したDataFrameをどうしますか？")
    print("1:別のcsvファイルデータと結合して新しいcsvファイルに保存  2:指定した値に上書きして新しいcsvファイルに保存  その他:そのまま新しいcsvファイルに保存")
    newdfse=yorn()
    if newdfse=="1" or newdfse=="2":
      print("未実装")
      continue
    else:
       # uname 保存するファイル名を入力させる　空白なら自動で名前を決定
      print("保存するファイルの名前を入力（末尾が'.csv'、空白で自動的に設定）>>")
      uname=yorn()
      if uname=="":
          a=1
          while emfil0(a)+".csv" in csv_list:
            a+=1
          uname=emfil0(a)+".csv"
      # url uname 指定ここまで
    # .csvでなければ修正(uname)
      while uname[-4:]!=".csv" or uname in csv_list:
        print("警告：保存するファイルの名前には未使用のcsvファイルを指定してください  1:現在の指定したファイル名に'.csv'を付け足す　その他:修正する")
        print("現在の指定ファイル名:"+uname)
        print("現在使われているファイル名")
        print(csv_list)
        unamere=input(">>")
        if unamere=="1":
          uname+=".csv"
        else:
          print("ファイル名を入力（末尾が'～.csv'）>>")
          uname=yorn()

      df3.to_csv("/content/"+uname)
    continue

  elif com[0]=="make":
    print("未実装")
    continue
  elif com[0]=="manual":
    text=""
    while text!="exit":
      if text!="run":
        print("コード入力  'run'で実行  'exit'で終了")
        text+=input(">>")
      else:
        exec(text)
        text==""
    continue
  elif com[0]=="del":
     # uname ファイル名を入力させる　空白なら自動で名前を決定
      print("削除ファイルの名前を入力（末尾が'.csv'、該当なしでキャンセル>>")
      print(csv_list)
      uname=yorn()
      if uname not in csv_list:
        print("該当なし キャンセル")
        continue
      else:
        os.remove("/content/"+uname)
        print("削除しました")
        continue
  else:
    print("エラー　不適切なコマンドです")

print("プログラム終了")





















コマンド入力...>>del
削除ファイルの名前を入力（末尾が'.csv'、該当なしでキャンセル>>
['sample_data/california_housing_train.csv', 'sample_data/mnist_train_small.csv', 'sample_data/mnist_test.csv', 'sample_data/california_housing_test.csv']
>>
:これで間違いありませんか？     y / other
該当なし キャンセル
コマンド入力...>>base
エラー
コマンド入力...>>base\housing_median_age
エラー
コマンド入力...>>ls
17000*9   sample_data/california_housing_train.csv
19999*785 sample_data/mnist_train_small.csv
9999*785  sample_data/mnist_test.csv
3000*9    sample_data/california_housing_test.csv
コマンド入力...>>base\sample_data/california_housing_test.csv
9列Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')
データを絞り込みます    1:数値の大小で比較　2:値の一致で比較　3:列名で指定した列を抽出  4:列名で指定した列を除いて抽出  その他:抽出終了
>>
:これで間違いありませんか？     y / other
抽出したDataFrameをどうしますか？
1:別のcsvファイルデータと結合して新しいcsvファイルに保存  2:指定した値に上書きして新しいcsvファイルに保存  その他:そのまま新しいcsvファイルに保存
>>
:これで間違いありませんか？     y / other
